In [5]:
import boto3

session = boto3.Session()
region = session.region_name
s3 = boto3.client("s3")

buckets = [b["Name"] for b in s3.list_buckets()["Buckets"]]
print(f"✅ Region: {region}")
print("✅ S3 Buckets:", buckets)


✅ Region: us-east-1
✅ S3 Buckets: ['crashtruth-frames', 'crashtruth-raw', 'crashtruth-reports', 'sagemaker-studio-993260645905-0hf2uesofvr9', 'sagemaker-studio-993260645905-ybwygpjbpw', 'sagemaker-us-east-1-993260645905', 'taxbuddy-embeddings-993260645905', 'taxbuddy-processed-993260645905', 'taxbuddy-raw-harthik-993260645905']


In [6]:
import boto3, sagemaker
from sagemaker.huggingface import HuggingFaceModel

region = boto3.Session().region_name
role = sagemaker.get_execution_role()

# YOLOS - lightweight transformer object detector (no timm dependency)
model = HuggingFaceModel(
    role=role,
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310",
    env={
        "HF_MODEL_ID": "hustvl/yolos-small",
        "HF_TASK": "object-detection"
    }
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge"
)

endpoint_name = predictor.endpoint_name
print("✅ YOLOS endpoint deployed:", endpoint_name)


-----------!✅ YOLOS endpoint deployed: huggingface-pytorch-inference-2025-10-26-02-21-39-496


In [7]:
import boto3, base64, json
from sagemaker.predictor import Predictor

FRAMES_BUCKET = "crashtruth-frames"
FRAME_KEY = "9833a2ca-5ba5-40a5-b5a2-0f4f65c7d09c/9833a2ca-5ba5-40a5-b5a2-0f4f65c7d09c.0000010.jpg"

s3 = boto3.client("s3")
img = s3.get_object(Bucket=FRAMES_BUCKET, Key=FRAME_KEY)["Body"].read()
b64 = base64.b64encode(img).decode("utf-8")

predictor = Predictor(endpoint_name="huggingface-pytorch-inference-2025-10-26-02-21-39-496")
payload = json.dumps({"inputs": b64}).encode("utf-8")

result = predictor.predict(payload, initial_args={"ContentType": "application/json"})
print(result)


b'[{"score":0.996321439743042,"label":"car","box":{"xmin":277,"ymin":171,"xmax":381,"ymax":269}},{"score":0.9117099642753601,"label":"car","box":{"xmin":383,"ymin":188,"xmax":407,"ymax":209}},{"score":0.9980326294898987,"label":"car","box":{"xmin":448,"ymin":182,"xmax":617,"ymax":273}},{"score":0.9896635413169861,"label":"car","box":{"xmin":230,"ymin":198,"xmax":255,"ymax":208}},{"score":0.9150543808937073,"label":"car","box":{"xmin":0,"ymin":4,"xmax":847,"ymax":471}},{"score":0.9931886196136475,"label":"car","box":{"xmin":104,"ymin":203,"xmax":165,"ymax":229}},{"score":0.9516823887825012,"label":"car","box":{"xmin":385,"ymin":189,"xmax":408,"ymax":208}}]'


In [8]:
import boto3, base64, json
from sagemaker.predictor import Predictor

FRAMES_BUCKET = "crashtruth-frames"
VIDEO_PREFIX  = "9833a2ca-5ba5-40a5-b5a2-0f4f65c7d09c/"
REPORTS_BUCKET = "crashtruth-reports"
REPORT_KEY     = f"{VIDEO_PREFIX.rstrip('/')}/detections_all.jsonl"

predictor = Predictor(endpoint_name="huggingface-pytorch-inference-2025-10-26-02-21-39-496")
s3 = boto3.client("s3")

# List all frames
frame_keys = []
continuation_token = None
while True:
    args = {"Bucket": FRAMES_BUCKET, "Prefix": VIDEO_PREFIX}
    if continuation_token:
        args["ContinuationToken"] = continuation_token
    resp = s3.list_objects_v2(**args)
    frame_keys.extend(
        [o["Key"] for o in resp.get("Contents", []) if o["Key"].lower().endswith(".jpg")]
    )
    if not resp.get("IsTruncated"):
        break
    continuation_token = resp.get("NextContinuationToken")

print(f"📸 Found {len(frame_keys)} frames to process")

lines = []
for i, k in enumerate(sorted(frame_keys), 1):
    img = s3.get_object(Bucket=FRAMES_BUCKET, Key=k)["Body"].read()
    b64 = base64.b64encode(img).decode("utf-8")
    out = predictor.predict(
        json.dumps({"inputs": b64}).encode("utf-8"),
        initial_args={"ContentType": "application/json"},
    )
    detections = json.loads(out.decode("utf-8")) if isinstance(out, (bytes, bytearray)) else out
    lines.append(json.dumps({"frame": k, "detections": detections}))
    if i % 25 == 0:
        print(f"Processed {i} frames...")

s3.put_object(
    Bucket=REPORTS_BUCKET,
    Key=REPORT_KEY,
    Body=("\n".join(lines)).encode("utf-8"),
    ContentType="application/json"
)

print(f"✅ All frame detections saved to: s3://{REPORTS_BUCKET}/{REPORT_KEY}")


📸 Found 313 frames to process
Processed 25 frames...
Processed 50 frames...
Processed 75 frames...
Processed 100 frames...
Processed 125 frames...
Processed 150 frames...
Processed 175 frames...
Processed 200 frames...
Processed 225 frames...
Processed 250 frames...
Processed 275 frames...
Processed 300 frames...
✅ All frame detections saved to: s3://crashtruth-reports/9833a2ca-5ba5-40a5-b5a2-0f4f65c7d09c/detections_all.jsonl
